<a href="https://colab.research.google.com/github/rishipython/Acute-Lymphocytic-Leukemia-ALL-Cell-Classification/blob/main/ALLCellClassification_Get_Mean_And_Std.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup (import packages, access GPU, etc.)

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
import tensorflow.keras as tfk
import tensorflow.keras.applications as tfa
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.vis_utils import plot_model
import json
import statistics as stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_dir = '/content/drive/MyDrive/'

In [ ]:
!ls /content/drive/MyDrive/

fold_0	fold_2		  test_y.npy	     train_y.npy      val_y.npy
fold_1	test_x_files.npy  train_x_files.npy  val_x_files.npy


In [ ]:
# Access GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Aug 11 01:31:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Access more memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# See if Tensorflow is using GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Get Mean and Standard Deviation of Training Set

In [ ]:
train_x_files = np.load(drive_dir+"train_x_files.npy")

In [ ]:
train_images = []

for i, file in enumerate(train_x_files):
  print(f"Images loaded: {i+1}/{len(train_x_files)}")
  train_images.append(imread(file))

train_images = np.array(train_images)

Streaming output truncated to the last 5000 lines.
Images loaded: 1415/6414
Images loaded: 1416/6414
Images loaded: 1417/6414
Images loaded: 1418/6414
Images loaded: 1419/6414
Images loaded: 1420/6414
Images loaded: 1421/6414
Images loaded: 1422/6414
Images loaded: 1423/6414
Images loaded: 1424/6414
Images loaded: 1425/6414
Images loaded: 1426/6414
Images loaded: 1427/6414
Images loaded: 1428/6414
Images loaded: 1429/6414
Images loaded: 1430/6414
Images loaded: 1431/6414
Images loaded: 1432/6414
Images loaded: 1433/6414
Images loaded: 1434/6414
Images loaded: 1435/6414
Images loaded: 1436/6414
Images loaded: 1437/6414
Images loaded: 1438/6414
Images loaded: 1439/6414
Images loaded: 1440/6414
Images loaded: 1441/6414
Images loaded: 1442/6414
Images loaded: 1443/6414
Images loaded: 1444/6414
Images loaded: 1445/6414
Images loaded: 1446/6414
Images loaded: 1447/6414
Images loaded: 1448/6414
Images loaded: 1449/6414
Images loaded: 1450/6414
Images loaded: 1451/6414
Images loaded: 1452/6414

In [ ]:
mean = np.mean(train_images)

In [ ]:
std = np.std(train_images)

In [ ]:
normalization_params = {'mean': mean, 'std': std}

In [ ]:
print(normalization_params)

{'mean': 9.854626920021918, 'std': 24.502705188655078}


In [ ]:
normalization_params_file = open(drive_dir+"normalization_params.json", "w")
json.dump(normalization_params, normalization_params_file)
normalization_params_file.close()